Encoder Training
==============

In [1]:
%pylab notebook

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import matplotlib as plt

import primo.models
import primo.datasets
import primo.tools.sequences as seqtools

from copy import deepcopy

from primo.models.cas9_keras import log10_crispr_spec, log10_norm_crispr_spec, linear_crispr_spec, dotproduct_crispr_spec, dotproduct_linearized

from primo.models.encoder import entropy_regularizer

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-25_blvqb because the default path (/tf/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Populating the interactive namespace from numpy and matplotlib
1 Physical GPUs, 1 Logical GPUs


In [2]:
train_dataset = primo.datasets.OpenImagesTrain(
    '/tf/open_images/train/', switch_every=10**5
)
validation_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')

# To see how this value was derived, please consult the Materials and Methods subsection under Feature Extraction section.
similarity_threshold = 75
# Intuitively determined:
batch_size = 150
val_batch_size = 150


def keras_batch_generator(dataset_batch_generator):
    # Yield datasets
    while True:
        # This tuple contains:
        # indices: a positive integer uniquely identifying an image. This index is obtained by enumerating all the images in the dataset (before splitting them into test/train/validate datasets)
        # pairs:
        indices, triplets = next(dataset_batch_generator)
        yield triplets, np.zeros(len(triplets))

train_batch_generator = keras_batch_generator(
    primo.datasets.dataset.triplet_batch_generator(
        train_dataset.random_features(batch_size),
        similarity_threshold
    )
)

val_batch_generator = keras_batch_generator(
    primo.datasets.dataset.triplet_batch_generator(
        validation_dataset.random_features(val_batch_size),
        similarity_threshold
    )
)


train_inputs, train_targets = next(train_batch_generator)
val_inputs, val_targets = next(val_batch_generator)

switching to train_9.h5 and train_1.h5


AttributeError: 'OpenImagesVal' object has no attribute 'path'

Reserve space on the GPU for running simulations. It's important to do this before running any tensorflow code (which will take all available GPU memory):

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Load up the training and validation datasets:

In [ ]:
train_dataset = primo.datasets.OpenImagesTrain(
    '/tf/open_images/train/', switch_every=10**5
)

validation_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')

In [ ]:
def keras_batch_generator(dataset_batch_generator, similarity_threshold):
    # Yield datasets
    while True:
        indices, pairs = next(dataset_batch_generator)
        # The Euclidean distances between the two vectors in each pair
        distances = np.sqrt(np.square(pairs[:,0,:] - pairs[:,1,:]).sum(1))
        # Whether or not the images in this pair should be considered 'similar'. This is a boolean value, represented by an int (0 or 1), and is determined by whether the aforementioned Euclidean distances between image feature vectors are under some pre-deterined "similarity threshold".
        similar = (distances < similarity_threshold).astype(int)
        # Yield a pair of sequences, and 0-or-1 indicating whether they're similar.
        yield pairs, similar

In [ ]:
# To see how this value was derived, please consult the 
# Materials and Methods subsection under Feature Extraction section in Bee et al. 2021
similarity_threshold = 75

# Intuitively determined:
encoder_training_dataset_batch_size = 100
# Intuitively determined:
encoder_validation_dataset_batch_size = 2500

encoder_train_batches = keras_batch_generator(
    train_dataset.balanced_pairs(encoder_training_dataset_batch_size, similarity_threshold),
    similarity_threshold
)

encoder_val_batches = keras_batch_generator(
    validation_dataset.random_pairs(encoder_validation_dataset_batch_size),
    similarity_threshold
)

predictor_train_batch_size = 1000
predictor_train_batches = train_dataset.random_pairs(predictor_train_batch_size)

Create the models and stack them together with the trainer:

In [ ]:
encoder = primo.models.Encoder('/tf/primo/data/models/encoder-function-P.h5')

# TODO: Replace the yield_predictor with the nucleaseq Cas9 predictor, use that here instead. https://github.com/uwmisl/cas9-similarity-search/issues/3 
#yield_predictor = primo.models.PredictorModel('/tf/primo/data/models/yield-model.h5')
yield_predictor = primo.models.PredictorFunction()
encoder.model.compile()
yield_predictor.model.compile()
encoder_trainer = primo.models.EncoderTrainer(encoder, yield_predictor)
encoder_trainer.model.summary()

Run the training!

In [ ]:
encoder_trainer.model.compile(tf.keras.optimizers.Adagrad(1e-3), 'binary_crossentropy')

In [ ]:
history = encoder_trainer.model.fit_generator(
    encoder_train_batches,
    steps_per_epoch = 1000,
    epochs = 100,
    validation_data = encoder_val_batches,
    validation_steps = 1,
    verbose = 2
)

Save the models:

In [ ]:
encoder.save('/tf/primo/data/models/encoder_model.h5')